In [ ]:
import pandas

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
BASE_DIR = '/content/drive/MyDrive/LLM causality/'
TABLES_DIR = BASE_DIR + 'Tables/'

In [ ]:
import re
import pandas as pd


def merge_model_data(models_table_path, capability_path):
    # Read the CSV files into pandas DataFrames
    models_table = pd.read_csv(models_table_path)
    capability = pd.read_csv(capability_path)

    # Process model names in capability.csv
    capability['Processed_Model'] = capability['fullname'].str.split('/').str[-1].str.lower()

    # Process model names in models_table.csv
    models_table['Processed_Model'] = models_table['Playground'].str.split('/').str[-1].str.lower()

    # Create all possible variations of model names in both dataframes
    def generate_variants(model_name):
        model_name = str(model_name)
        variants = set()
        variants.add(model_name)
        variants.add(model_name.replace("-", " "))
        # Handle cases where some hyphens are replaced and some are not
        for i in range(model_name.count("-")):
          new_name = model_name
          for j in range(i):
            new_name = new_name.replace("-", " ", 1)
          variants.add(new_name)
        return variants

    capability['Model_Variants'] = capability['Processed_Model'].apply(generate_variants)
    models_table['Model_Variants'] = models_table['Processed_Model'].apply(generate_variants)


    #Explode the variants columns to create rows for each possible variant
    capability = capability.explode("Model_Variants")
    models_table = models_table.explode("Model_Variants")

    # Merge the DataFrames based on the processed model names using all possible variants
    merged_df = pd.merge(capability, models_table, on='Model_Variants', how='inner')

    # Select the desired columns and rename the 'Model_x' column to 'Model'
    merged_df = merged_df[['Model_x'] + [col for col in merged_df.columns if col not in ['Model_x', 'Model_y', 'Processed_Model_x', 'Processed_Model_y', "Model_Variants_x", "Model_Variants_y"]]]
    merged_df = merged_df.rename(columns={'Model_x': 'Model'})

    return merged_df

# Example usage (replace with your actual file paths):
merged_data = merge_model_data(TABLES_DIR + 'models_table.csv', TABLES_DIR + 'open_llm_leaderboard.csv')
len(merged_data)


164

In [ ]:
# Remove duplicate rows based on the 'Model' column
merged_data = merged_data.drop_duplicates(subset=['Model'])

# Save the filtered data to a new CSV file
merged_data.to_csv(TABLES_DIR + 'models_table_filtered.csv', index=False)
